In [ ]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from datasets import load_dataset
import cassio
from PyPDF2 import PdfReader
from typing_extensions import Concatenate


In [13]:
from dotenv import load_dotenv
import os

load_dotenv()  # This will look for a .env file in the current directory

ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")
GROQ_TOKEN = os.getenv("GROQ_TOKEN")
HF_TOKEN = os.getenv("HF_TOKEN")

os.environ['HF_TOKEN'] = HF_TOKEN

In [22]:
embedding=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
llm=ChatGroq(groq_api_key=GROQ_TOKEN,model_name='llama-3.1-8b-instant')

In [26]:
pdfreader = PdfReader('data/sr117.pdf')
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [27]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [28]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="mrm_policy",
    session=None,
    keyspace=None,
)

In [29]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [30]:

astra_vector_store.add_texts(texts)

print("Inserted %i headlines." % len(texts))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 115 headlines.


In [32]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content))


QUESTION: "In IRC model, what is the simulation horizon"
ANSWER: "The simulation horizon in an IRC (Internal Ratings-Based) model is not explicitly mentioned in the provided context. However, it is implied that the model is built for a one-year capital horizon (mentioned in points 2, 15, and 16), which aligns with the 99.9 percent soundness standard over a one-year capital horizon as described in the Basel II Framework.

If you are looking for more information on the simulation horizon in an IRC model, I would recommend consulting the Basel II Framework or other relevant regulatory documents for more detailed information."

FIRST DOCUMENTS BY RELEVANCE:
    [0.7883] "14. As described immediately below, for each IRC-covered position the model should 
also capture the impact of rebalancing positions at  the end of their liquidity horizons so as to 
achieve a constant level of risk over a one- year capital horizon. The model may incorporate 
correlation effects among the modelled risk fa